<h1 style="text-align: center;">Taxi-v3</h1>

In [ ]:
from IPython.display import clear_output

import gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

In [ ]:
# Environment
exercise = 'Taxi-v3'

# Hyperparameter
alpha = .1
epsilon = .1
gamma = .95
episodes = 10000

# Visualization
rolling_mean_size = 1
marker = ','
line = '-'

In [ ]:
def choose_action_epsilon_greedy(state, epsilon, q_map, actions):
    if random.uniform(0, 1) <= epsilon:
        return random.randrange(0, actions)
    else:
        return np.argmax(q_map[state])


def choose_action_greedy(state, q_map, actions):
    return choose_action_epsilon_greedy(state, 0, q_map, actions)


def plot_episode_statistics(data, episodes, label):
    df = pd.DataFrame({'x': np.arange(1, episodes + 1), 'y': data})
    rolling_means = df.rolling(rolling_mean_size).mean()
    plt.plot('x', 'y', f'{marker}{line}', data=rolling_means, label=label)

<h2 style="text-align: center;">Dyna-Q</h2>
The Dyna-Q algorithm is an extension of the Q-Learnings algorithm with a planning phase. 
During the execution of the episodes we create an internal MDP from the observed behaviour.
That MDP has two functions $ \hat{P}_{s, s'}^a $ and $ \hat{R}_s^a $.
$ \hat{P}_{s, s'}^a $ is the probability, that action $ a $ choosen in state $ s $ leeds to state $ s' $.
$ \hat{R}_s^a $ represents the expected reward if actionn $ a $ is choosen in step $ s $.

These two functions are maintained after each step in the environment.
After each step we execute $n $ planning steps where we choose a random state action pair that we observed earlier. 
With our created MDP we simulate a transition from $ s $ with action $ a $ and update the q-values with the gained reward.

That planning phase has the advantage that less episodes are needed to find the <i>real</i> Q-values.

<h2 style="text-align: center;">Dyna-Q+</h2>
Dyna-Q+ is an extension of the Dyna-Q algotithm to enable the agent to learn changes in a non-stationary environment.
The difference is that we add an exploration bonus to the received reward if the transition wasn't executed for a long time.
The algorithms stores the information when a transition was executed the last time. 
While updating the q-values we clculate the following values and add it to the received reward:
$$ \kappa*\sqrt{\tau} $$.
$ \kappa $ is a small constant factor. 
$ \tau $ is the number of steps that have been executed since the transitions was executed the last time.


In [ ]:
# Dyna-Q(+)
n = 5 # planning steps
kappa = .001

In [ ]:
def dyna_q_plus(exercise, number_of_episodes, epsilon, gamma, alpha, planning_steps, kappa):
    env = gym.make(exercise)
    steps_per_episode = []

    # Initialization
    q_map = np.zeros((env.nS, env.nA))

    transition_probabilities = np.zeros((env.nS, env.nS, env.nA))
    transition_rewards = np.zeros((env.nS, env.nA))
    state_action_state_transitions = np.zeros((env.nS, env.nS, env.nA))
    state_action_transitions = np.zeros((env.nS, env.nA))
    state_action_rewards = np.zeros((env.nS, env.nA))

    # Dyna-Q+ Extension
    last_visit = np.zeros((env.nS, env.nA))
    visited_states = set()

    global_step_counter = 0
    for episode in range(1, number_of_episodes + 1):
        episode_finished = False
        s = env.reset()
        step_counter = 0

        while not episode_finished:
            step_counter += 1
            global_step_counter += 1
            
            a = choose_action_epsilon_greedy(s, epsilon, q_map, env.nA)

            s_next, reward, done, _ = env.step(a)
            episode_finished = done

            a_next = choose_action_greedy(s_next, q_map, env.nA)

            # Dyna-Q+ Extension
            if last_visit[s, a] != 0:
                reward = reward + kappa * np.sqrt(global_step_counter - last_visit[s, a])

            q_map[s, a] = q_map[s, a] + alpha * (reward + gamma * q_map[s_next, a_next] - q_map[s, a])

            last_visit[s, a] = global_step_counter

            # Update model
            state_action_transitions[s, a] = state_action_transitions[s, a] + 1
            state_action_state_transitions[s, s_next, a] = state_action_state_transitions[s, s_next, a] + 1
            state_action_rewards[s, a] = state_action_rewards[s, a] + reward
            
            list_of_next_states = state_action_state_transitions[s, :, a]
            for element in range(len(list_of_next_states)):
                transition_probabilities[s, element, a] = state_action_state_transitions[s, element, a] / state_action_transitions[s, a]
            
            transition_rewards[s, a] = state_action_rewards[s, a] / state_action_transitions[s, a]
            visited_states.add((s, a))

            # Planning
            for n in range(planning_steps):
                (s_learn, a_learn) = random.choice(list(visited_states))

                target_states = transition_probabilities[s_learn, :, a_learn]

                s_learn_next = np.random.choice(np.arange(target_states.size), p=target_states)
                r_learn = transition_rewards[s_learn, a_learn]

                # Dyna-Q+ Extension
                r_learn = r_learn + kappa * np.sqrt(global_step_counter - last_visit[s_learn, a_learn])

                a_learn_next = choose_action_greedy(s_learn_next, q_map, env.nA)
                q_map[s_learn, a_learn] = q_map[s_learn, a_learn] + alpha * (
                            r_learn + gamma * q_map[s_learn_next, a_learn_next] - q_map[s_learn, a_learn])
            s = s_next
        steps_per_episode.append(step_counter)
    return steps_per_episode


In [ ]:
plt.figure(figsize=(20, 8))

plot_episode_statistics(dyna_q_plus(exercise, episodes, epsilon, gamma, alpha, 0, 0), episodes, f'Q-Learning')
plot_episode_statistics(dyna_q_plus(exercise, episodes, epsilon, gamma, alpha, n, 0), episodes, f'Dyna-Q: {n} planning steps')
plot_episode_statistics(dyna_q_plus(exercise, episodes, epsilon, gamma, alpha, n, kappa), episodes, f'Dyna-Q+: {n} planning steps')

plt.legend()
plt.title('Comparing Algorithms')
plt.xlabel('Episodes')
plt.ylabel('Steps per episode')
plt.show()

<h2 style="text-align: center;">Simple Monte-Carlo Search</h2>
From the current state $ s $ we generate $ k $ episodes with the learned model.
Afterwards we update the q-values from $ s, a $ with the mean reward from the generated episode. 

In [ ]:
# Simple Monte Carlo Search
k = 10 # simulated episodes
max_episode_length = 200

In [ ]:
def simple_monte_carlo_search(exercise, number_of_episodes, epsilon, gamma, alpha, simulated_episodes):
    env = gym.make(exercise)
    steps_per_episode = []
    
    # Initialization
    q_map = np.zeros((env.nS, env.nA))
    
    transition_probabilities = np.zeros((env.nS, env.nS, env.nA))
    transition_rewards = np.zeros((env.nS, env.nA))
    state_action_state_transitions = np.zeros((env.nS, env.nS, env.nA))
    state_action_transitions = np.zeros((env.nS, env.nA))
    state_action_rewards = np.zeros((env.nS, env.nA))
    visited_states = set()
    
    final_states = set()
    
    for e in range(1, number_of_episodes + 1):
        episode_finished = False
        s = env.reset()
        step_counter = 0
    
        while not episode_finished:
            step_counter += 1
            a = choose_action_epsilon_greedy(s, epsilon, q_map, env.nA)
            
            s_next, reward, done, _ = env.step(a)
            
            if done and step_counter != 200:
                final_states.add(s_next)
            
            episode_finished = done
            
            a_next = choose_action_greedy(s_next, q_map, env.nA)
            
            q_map[s, a] = q_map[s, a] + alpha * (reward + gamma * q_map[s_next, a_next] - q_map[s, a])
            
            # Update model
            state_action_transitions[s, a] = state_action_transitions[s, a] + 1
            state_action_state_transitions[s, s_next, a] = state_action_state_transitions[s, s_next, a] + 1
            state_action_rewards[s, a] = state_action_rewards[s, a] + reward
            
            list_of_next_states = state_action_state_transitions[s, :, a]
            for element in range(len(list_of_next_states)):
                transition_probabilities[s, element, a] = state_action_state_transitions[s, element, a] / state_action_transitions[s, a]
            
            transition_rewards[s, a] = state_action_rewards[s, a] / state_action_transitions[s, a]
            visited_states.add((s, a))
            
            # Planning
            for a_index in range(env.nA):
                if state_action_transitions[s, a_index] == 0:
                    continue
                for k in range(1, simulated_episodes+1):
                    episode = simulate_full_episode(s_next, a_index, epsilon, final_states, transition_rewards, transition_probabilities, q_map)
                    g = 0
                    for t in reversed(range(len(episode))):
                        step = episode[t]
                        state = step[0]
                        reward = step[2]
                        g = gamma * g + reward
                    q_map[s, a] = q_map[s, a] +  (1/simulated_episodes+1) * (g - q_map[s, a])
                
            s = s_next
        steps_per_episode.append(step_counter)
    return steps_per_episode

def simulate_full_episode(state, action, epsilon, final_states, transition_rewards, transition_probabilities, q_map):
    episode = []
    while True:
        reward = transition_rewards[state, action]
        
        # Mögliche Zielzustände
        target_states = transition_probabilities[state, :, action]
        if not np.any(target_states):
            break
        
        s_next = np.random.choice(np.arange(target_states.size), p = target_states)
        
        episode.append([state, action, reward])
        
        # Nächsten State setzen
        state = s_next
        
        # Nächste Action setzen
        target_actions = q_map[state]
        if not np.any(target_actions):
            break
        
        if random.uniform(0, 1) <= epsilon:
            action = random.randrange(0, 5)
        else:
            # Tie-breaking argmax
            action = np.random.choice(np.flatnonzero(target_actions == target_actions.max()))
        
        if state in final_states:
            episode.append([state, -1, 0])
            break
            
        if len(episode) == max_episode_length:
            break
    return episode

In [ ]:
plt.figure(figsize=(20, 8))

plot_episode_statistics(simple_monte_carlo_search(exercise, episodes, epsilon, gamma, alpha, k), episodes, f'Simple Monte Carlo Search: {k} episode simulations')

plt.legend()
plt.title('Comparing Algorithms')
plt.xlabel('Episodes')
plt.ylabel('Steps per episode')
plt.show()